<a href="https://colab.research.google.com/github/Mbaiornom/Modelisation_entraintement_R_Python/blob/main/Fernand_stage_SNCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importation des bibliothèques necessaires

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.feature_selection import SelectKBest, f_classif
import matplotlib.pyplot as plt
!pip install pandas matplotlib seaborn
import seaborn as sns

Connection au drive

In [7]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


Importation de la base des données

In [15]:
base_2023 = pd.read_csv('/content/drive/MyDrive/Stage_SNCF/Baro_SNCF_2023_Data_2023.csv',sep=';')

Base_2023 = pd.DataFrame(base_2023)

#print(Base_2023)

Vue globale de la base

In [16]:
Base_2023.describe(include="all")

,ID BDD,ANNEE,SEXE,CLASSE,L1,L2,L3,Q1,Q2,Q3,...,ETABLISSEMENT_394,ETABLISSEMENT_395,ETABLISSEMENT_396,ETABLISSEMENT_397,ETABLISSEMENT_398,ETABLISSEMENT_399,ETABLISSEMENT_400,ETABLISSEMENT_401,ETABLISSEMENT_402,ETABLISSEMENT_403
count,99123.000000,99123.0,99123.000000,99123.000000,99123.0,99089.000000,98599.000000,99123.000000,99123.000000,99123.000000,...,99123.000000,99123.000000,99123.000000,99123.000000,99123.000000,99123.000000,99123.000000,99123.000000,99123.000000,99123.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,49562.000000,2023.0,1.247511,4.704690,1.0,3.912372,26.099737,2.811456,1.966637,2.111911,...,0.005085,0.002058,0.005750,0.006487,0.005771,0.003914,0.006941,0.007657,0.003027,0.030013
std,28614.489704,0.0,0.452452,2.409776,0.0,1.163923,11.492733,1.407862,0.804355,0.887861,...,0.071125,0.045319,0.075614,0.080280,0.075745,0.062442,0.083023,0.087170,0.054931,0.170624
min,1.000000,2023.0,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24781.500000,2023.0,1.000000,3.000000,1.0,3.000000,18.000000,1.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,49562.000000,2023.0,1.000000,4.000000,1.0,4.000000,32.000000,3.000000,2.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,74342.500000,2023.0,1.000000,6.000000,1.0,5.000000,34.000000,4.000000,2.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Transformation de la base: création de la variable etablissement

In [17]:
# Identifier les colonnes d'établissements

etablissement_columns = [col for col in Base_2023.columns if col.startswith('ETABLISSEMENT_')]

# Créer une nouvelle colonne "Etablissement"

Base_2023['ETABLISSEMENT'] = Base_2023[etablissement_columns].idxmax(axis=1)

# Extraire le numéro de l'établissement

Base_2023['ETABLISSEMENT'] = Base_2023['ETABLISSEMENT'].apply(lambda x: int(x.split('_')[1]))

# Supprimer les colonnes d'établissements initiales

#Base_2023 = Base_2023.drop(columns=etablissement_columns)

# Afficher le DataFrame avec la nouvelle colonne "Etablissement"
print("\nDataFrame avec la colonne 'Etablissement':")
print(Base_2023)


DataFrame avec la colonne 'Etablissement':
       ID BDD  ANNEE  SEXE  CLASSE  L1   L2    L3  Q1  Q2  Q3  ...  \
0           1   2023     1       4   1  5.0  32.0   4   3   3  ...   
1           2   2023     1       2   1  5.0  34.0   1   1   1  ...   
2           3   2023     1       4   1  5.0  32.0   3   2   3  ...   
3           4   2023     1       5   1  5.0  34.0   3   2   2  ...   
4           5   2023     1       2   1  5.0  32.0   1   1   1  ...   
...       ...    ...   ...     ...  ..  ...   ...  ..  ..  ..  ...   
99118   99119   2023     1      15   1  2.0   1.0   1   1   1  ...   
99119   99120   2023     1       6   1  3.0  36.0   4   1   1  ...   
99120   99121   2023     2       5   1  5.0  34.0   3   2   2  ...   
99121   99122   2023     2       1   1  4.0  24.0   3   2   1  ...   
99122   99123   2023     1       3   1  4.0  24.0   2   1   1  ...   

       ETABLISSEMENT_395  ETABLISSEMENT_396  ETABLISSEMENT_397  \
0                      0                  0      

Renommer les noms des colonnes

In [20]:
Base_2023_renamed = Base_2023.rename(columns={
'L1':'SNCF_Group','L2':'Niveau SA','L3':'Niveau 3','Q1':'etat_desprit','Q2':'aimer_travailler_pour_SNCF','Q3':'etre_fier_de_travailler_pour_SNCF',
'Q4':'recommander_SNCF','Q5':'confiance_ensonavenir_SNCF','Q6A':'Posibilite_evolution_ausein_SNCF','Q6':'satisfaction_ambiance_ausein_equipe','Q7':'satisfaction_environnement_travail','Q8':'satisfaction_equilibre_travail_vie_personnelle',
'Q9_1_1':'satisfaction_moyens_materiels','Q9_2_2':'satisfaction_moyens_humains','Q9_3_3':'satisfaction_moyens_autonomie_et_marges_manoeuvres',
'Q9_4_4':'satisfaction_outils_informatiques','Q9_5_5':'satisfaction_soutient_descollegues','Q9_6_6':'satisfaction_soutient_mon_manage',
'Q9_7_7':'satisfaction_acces_informations_metiers','Q9_8_8':'satisfaction_temps','Q10A':'irritants_quotidien_traites','Q10':'benefier_informations_necessaires_montravail',
'Q11':'encourager_apartager_bonnes_idees_avec_collectif','Q12':'reussites_reconnu_etvaloriseesomonequipe','Q13':'travail_adu_sens_pour_moi','Q13A_01':'premiere_raison_tsprmoi',
'Q13A_02':'deuxieme_raison_tsprmoi','Q13A_03':'troisieme_raison_tsprmoi','Q13B_01':'premiere_raison_tvl_pas_sensprmoi','Q13B_02':'deuxieme_raison_tvl_pas_sensprmoi','Q13B_03':'troisieme_raison_tvl_pas_sensprmoi',
'Q14_01':'premier_sujet_prioritaire_desconditions_detravail','Q14_02':'deuxieme_sujet_prioritaire_desconditions_detravail',
'Q15':'effort_contribution_resultats_reconnues_parmanager','Q16':'comprendre_montravail_contribue_objectifs_equipe','Q17':'echanges_reguliers_manager_pointfs_fortsetfaibles','Q18':'monmanager_ttentif_auxretours','Q19':'monmanager_valorise','Q20_1_1':'sentiment_etretrate_equitable_Promotion_professionnelle',
'Q20_2_2':'sentiment_etretrate_equitable_remuneration','Q20_3_3':'sentiment_etretrate_equitable_repartition_charge_tvl','Q20_4_4':'sentiment_etretrate_equitable_dvpmt_competences','Q21_01':'premier_point_fort_avec_monmanager',
'Q21_02':'deuxieme_point_fort_avec_monmanager','Q21BIS_01':'premiere_amelioration_attendue_avec_monmanager','Q21BIS_02':'deuxieme_amelioration_attendue_avec_monmanager','Q22':'encadrez_vous_uneouplusieurs_personnes','Q23_1_1':'entantquemanager_satisfaction_moyens_humains',
'Q23_2_2':'entantquemanager_satisfaction_moyens_materiels','Q23_3_3':'entantquemanager_satisfaction_appui_accompagnement_devotrehierarch','Q23_4_4':'entantquemanager_assistance_accompagenement_RH','Q23_5_5':'entantquemanager_satisfaction_marges_manoeuvre','Q23_6_6':'entantquemanager_satisfaction_informations_menerabienses_missions',
'Q23_7_7':'entantquemanager_satisfaction_formation_encadrement_equipe','Q23_8_8':'entantquemanager_satisfaction_temps_management_desquipes'
})
# Afficher le DataFrame avec les colonnes renommées
print("\nDataFrame avec colonnes renommées:")
print(Base_2023_renamed.head(5))


DataFrame avec colonnes renommées:
   ID BDD  ANNEE  SEXE  CLASSE  SNCF_Group  Niveau SA  Niveau 3  etat_desprit  \
0       1   2023     1       4           1        5.0      32.0             4   
1       2   2023     1       2           1        5.0      34.0             1   
2       3   2023     1       4           1        5.0      32.0             3   
3       4   2023     1       5           1        5.0      34.0             3   
4       5   2023     1       2           1        5.0      32.0             1   

   aimer_travailler_pour_SNCF  etre_fier_de_travailler_pour_SNCF  ...  \
0                           3                                  3  ...   
1                           1                                  1  ...   
2                           2                                  3  ...   
3                           2                                  2  ...   
4                           1                                  1  ...   

   ETABLISSEMENT_395  ETABLISSEMENT_39